###### Setting up Environment variables

In [1]:
import sys
sys.path.append('/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/')
sys.path.append('./')
print(sys.version)

3.10.11 (main, May 16 2023, 00:28:57) [GCC 11.2.0]


In [2]:
###########import necessary keys 

import os
#import openai
from secret import OPENAI_API_TYPE,OPENAI_API_KEY,OPENAI_API_BASE,HUGGINGFACEHUB_API_TOKEN
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.llms import OpenAIChat
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv()

# openai.api_version = "2023-07-01-preview"

os.environ['OPENAI_API_TYPE']=OPENAI_API_TYPE
os.environ['OPENAI_API_BASE']=OPENAI_API_BASE
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN






###### Set & Identify right LLM Model & Langchain Module 

In [3]:
##### Unstable connectivity between langchain and openai. Better to test which link is working an any given point in time. 

os.environ['OPENAI_API_VERSION'] = "2023-08-01-preview" #"2023-05-15" #"2022-12-01"
deployment_list =["deploy-name-embed","deploy-gpt35turbo16k", "datachatdeploy"]
model_list=["text-embedding-ada-002","gpt-35-turbo-16k", "text-davinci-003","gpt-35-turbo"]

for deploy_name in deployment_list:
    for mod_name in model_list:
        try:
            llm = AzureOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAI")
                break
        except Exception as e:
            print(f"Error with AzureOpenAI:{e}\n" )
        
        try:
            llm = AzureChatOpenAI(deployment_name=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : AzureOpenAIChat")
                break
        except Exception as e:
            print(f"Error with AzureOpenAIChat:{e}\n" )
        
        try:
            llm= OpenAI(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAI")
                break
        except Exception as e:
            print(f"Error with OpenAI:{e}\n" )

        try:
            llm= OpenAIChat(engine=deploy_name,model_name=mod_name,temperature=0)
            print(llm)
            if llm("what is esg?"):
                print("Successful : OpenAIChat")
                break
        except Exception as e:
            print(f"Error with OpenAIChat:{e}\n" )




AzureOpenAI
Params: {'deployment_name': 'deploy-name-embed', 'model_name': 'text-embedding-ada-002', 'temperature': 0.0, 'max_tokens': 256, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}
Error with AzureOpenAI:The completion operation does not work with the specified model, text-embedding-ada-002. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.

cache=None verbose=False callbacks=None callback_manager=None tags=None metadata=None client=<class 'openai.api_resources.chat_completion.ChatCompletion'> model_name='text-embedding-ada-002' temperature=0.0 model_kwargs={} openai_api_key='ff0b3bcd1b9843e2bf0c172b531cb52a' openai_api_base='https://ws-openai-ml-aoai.openai.azure.com/' openai_organization='' openai_proxy='' request_timeout=None max_retries=6 streaming=False n=1 max_tokens=None tiktoken_model_name=No

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/utils/utils.py:157: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain/llms/openai.py:787: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [4]:
print(llm)

OpenAIChat
Params: {'model_name': 'text-embedding-ada-002', 'engine': 'datachatdeploy', 'temperature': 0}


In [6]:
llm("what is esg?") 

"ESG stands for Environmental, Social, and Governance. It is a framework used to evaluate the sustainability and ethical impact of a company or investment. \n\nEnvironmental factors refer to a company's impact on the environment, such as its carbon emissions, waste management, and resource usage. Social factors consider a company's impact on society, including its treatment of employees, community engagement, and diversity and inclusion practices. Governance factors assess the company's leadership, board structure, executive compensation, and transparency in decision-making.\n\nESG criteria are used by investors, financial institutions, and stakeholders to assess the long-term sustainability and ethical practices of a company. It helps them make informed decisions about investments, as well as encourages companies to adopt responsible business practices."

#### Load the document file that will be vectorized and indexed.

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

loader = PyPDFLoader("./data/files/esg_strategy.pdf")
pages = loader.load_and_split() ## this split the doc by pdf pages.

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)


In [30]:
print("size of the doc, that is elements :",len(pages))
print(pages[0])
print()
print(pages[1])

size of the doc, that is elements : 5
page_content="Impact  of ESG Strategy  on Investment  Management  and Asset  Management:  A Case  Study  \non Northern  Trust  \nExecutive  Summary:  Environmental, Social, and Governance (ESG) factors have \nsignificantly influenced the investment landscape in recent years. For investment \nmanagement and asset management companies, including Northern Trust, the \nincorporation of ESG criteria can drive busi ness decisions, shape investment portfolios, \nand influence stakeholder relations. This document delves into the impact of ESG \nstrategies on these institutions.  \n1. Introduction:  \n• ESG Strategy : A set of investment criteria that consider environmental, social, \nand governance factors alongside financial factors in the investment decision -\nmaking process.  \n• Northern  Trust : An international investment management and asset \nmanagement company that has integrated ESG strategies into its products and \nservices.  \n2. Direct  Impa

In [29]:
# this split doc by lines or sentences.
from langchain.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("data/esg_strategy.pdf",mode='elements') # this split doc by lines or sentences.
data = loader.load()
print("size of the doc, that is elements :",len(data))
print(data[0])
print()
print(data[1])



size of the doc, that is elements : 102
page_content='Impact of ESG Strategy on Investment Management and Asset Management: A Case Study on Northern Trust' metadata={'source': 'data/esg_strategy.pdf', 'coordinates': {'points': ((72.264, 90.65840000000003), (72.264, 117.01999999999998), (541.70704, 117.01999999999998), (541.70704, 90.65840000000003)), 'system': 'PixelSpace', 'layout_width': 612, 'layout_height': 792}, 'filename': 'esg_strategy.pdf', 'file_directory': 'data', 'last_modified': '2023-09-19T14:56:32', 'filetype': 'application/pdf', 'page_number': 1, 'category': 'UncategorizedText'}

page_content='Executive Summary: Environmental, Social, and Governance (ESG) factors have significantly influenced the investment landscape in recent years. For investment management and asset management companies, including Northern Trust, the incorporation of ESG criteria can drive business decisions, shape investment portfolios, and influence stakeholder relations. This document delves into t

###### Handling multiple pdf,docs& text files for embeddings

In [ ]:
## handle multiple documents
from langchain.document_loaders import Docx2txtLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

documents = []
for file in os.listdir('docs'):
    if file.endswith('.pdf'):
        pdf_path = './docs/' + file
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = './docs/' + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        text_path = './docs/' + file
        loader = TextLoader(text_path)
        documents.extend(loader.load())

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

# we now proceed as earlier, passing in the chunked_documents to the
# to the vectorstore
# ...

In [35]:
embeddings = OpenAIEmbeddings(deployment="deploy-name-embed",
    model="text-embedding-ada-002",
    openai_api_base=OPENAI_API_BASE,
    openai_api_type="azure",chunk_size=16,)

print(embeddings)

In [32]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = DirectoryLoader('./data/files/',  show_progress=True,use_multithreading=True) ### parameter to control which files: glob="**/*.pdf",
docs=loader.load()

print(len(docs))

text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=10)
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0)
chunked_documents = text_splitter.split_documents(docs)


100%|██████████| 3/3 [00:25<00:00,  8.59s/it]


3


In [42]:
returned_embeddings = embeddings.embed_documents(
    [doc.page_content for doc in chunked_documents],
)


doc_embed_db = FAISS.from_embeddings(
    text_embeddings=[
        (doc.page_content, embedding)
        for doc, embedding in zip(chunked_documents, returned_embeddings)
    ], 
    embedding=embeddings,
    metadatas=[
        doc.metadata
        for doc in chunked_documents
    ],
)

query="what is Northern Trust approach for ESG?"

#query_result = embeddings.embed_query(query)
#query_result ### returns vector embeddings

In [43]:
## Saving the embeddings 

doc_embed_db.save_local("data/faiss_index")
#new_db = FAISS.load_local("faiss_index", embeddings)

In [63]:
### For retrieving the document 
##retriever = FAISS.from_documents(texts, embeddings).as_retriever()

##docs = retriever.get_relevant_documents("what is Northern Trust approach for ESG?")
##pretty_print_docs(docs)

Document 1:

execution of their ESG strategies. Through these questions, Northern Trust ensures that 
its investment decisions align with sustainability and responsible business practices, 
reflecting the evolving demands of modern investors.
----------------------------------------------------------------------------------------------------
Document 2:

portfolio optimization and client growth, as well as indirect benefits like enhanced 
reputation and stakeholder engagement.  
By understanding and actively incorporating ESG strategies, investment and asset 
management companies position themselves for sustainable growth and long -term 
success in a rapidly evolving financial landscape.  
 
 
Northern  Trust's  ESG Inquiry  Framework  for Companies  
Executive  Summary:  Northern Trust actively engages with companies to assess their 
adherence to and integration of Environmental, Social, and Governance (ESG) principles. 
This document provides an outline of the typical questions North

In [46]:
doc_embed_db.similarity_search(query)

[Document(page_content='Is the company actively working to decrease the frequency of health and safety issues?\n\nKey Benefits\n\n1\n\nEnables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.\n\n2\n\nProvides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.\n\n3\n\nOffers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.\n\n4\n\nAssists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metr

In [51]:
## compression of results to address output token limit issue
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

retriever =doc_embed_db.as_retriever()
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)


Document 1:

Enables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.
Provides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.
Offers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.
Assists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metric.
Best-in-class ESG is industry terminology referring to an investment approach that selects companies that are leaders in implementing ESG.
---

In [63]:
# EmbeddingsFilter
# Making an extra LLM call over each retrieved document is expensive and slow. The EmbeddingsFilter provides a cheaper and faster option by embedding the documents and query and only returning those documents which have sufficiently similar embeddings to the query.

from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter
import util.helper as helper



embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(query)
helper.pretty_print_docs(compressed_docs)

Document 1:

Is the company actively working to decrease the frequency of health and safety issues?

Key Benefits

1

Enables more purposeful and transparent integration of ESG considerations into investment processes, addressing the need for a consistent way to measure and report on ESG investments.

2

Provides a consistent, transparent methodology to gain greater clarity when building and managing sustainable portfolios by focusing on the magnitude and direction of key ESG-related business issues likely to have a financial impact on companies.

3

Offers more precision in identifying ESG-related business risks that could impact its performance, whether constructing portfolios using best-in- class security selection or exclusionary techniques.

4

Assists in prioritizing corporate engagements, further aligning sustainable investments and stewardship — leading to more deliberate engagements and the ability to track progress over time with a consistent metric.

Best-in-class ESG is ind

###### Adding Chat History

```
ConversationalRetrievalChain


from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(),
    vectordb.as_retriever(search_kwargs={'k': 6}),
    return_source_documents=True
)
```

#### CSV Data handling

In [10]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path="./data/files/esgdatakaggle/ESGCountry.csv",
    csv_args={
        "delimiter": ",",
        "quotechar": '"',  
        #"fieldnames": ["Country Code","Short Name","Table Name","Long Name","2-alpha code","Currency Unit","Special Notes","Region","Income Group","WB-2 code","National accounts base year","National accounts reference year","SNA price valuation","Lending category","Other groups","System of National Accounts","Alternative conversion factor","PPP survey year","Balance of Payments Manual in use","External debt Reporting status","System of trade","Government Accounting concept","IMF data dissemination standard","Latest population census","Latest household survey","Source of most recent Income and expenditure data","Vital registration complete","Latest agricultural census","Latest industrial data","Latest trade data"],
    },
)

data = loader.load()

#for document in data:
#    content = document.page_content
#    metadata = document.metadata


In [17]:
from langchain.prompts import PromptTemplate


ESG_TOPIC_CAPTURE_PROMPT = """
You are tasked to extract ESG (Environmental, Social, and Governance) topics for each country records. You will be provided with country name {country_name}  and text {text}.
ESG represents a broad spectrum of topics that investors and countries consider as part of their responsibility towards stakeholders and the broader environment. 

Please respond with list ESG topics names based on the topics above

YOUR RESPONSE:"""


ESG_TOPIC_CAPTURE_PROMPT_TEMPLATE = PromptTemplate(template=ESG_TOPIC_CAPTURE_PROMPT, input_variables=["text", "country_name"])

In [18]:
ESG_TOPIC_CAPTURE_FREQ_PROMPT = """
You are a helpful AI bot that aids a user in research.
You will be given a list of ESG topics about for a country {country_name}.

Please consolidate and provide number of times each topic discussed.

% ESG TOPICS
{text}
"""
ESG_TOPIC_CAPTURE_FREQ_PROMPT_TEMPLATE = PromptTemplate(template=ESG_TOPIC_CAPTURE_FREQ_PROMPT, input_variables=["text", "country_name"])

In [19]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm,
                             chain_type="map_reduce",
                             map_prompt=ESG_TOPIC_CAPTURE_PROMPT_TEMPLATE,
                             combine_prompt=ESG_TOPIC_CAPTURE_FREQ_PROMPT_TEMPLATE,
                            # verbose=True
                            )

In [14]:
doc_txt=""" 
ARG	IT.NET.USER.ZS	YR2000	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2001	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2002	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2003	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2004	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2005	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2006	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2007	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2008	Source: SECOM.
ARG	IT.NET.USER.ZS	YR2009	Source: Comisión Nacional de Comunicaciones
ARG	IT.NET.USER.ZS	YR2010	Source: ITU estimate.
ARG	IT.NET.USER.ZS	YR2011	Source: ITU estimate based on INDEC's survey.;Note: Population aged 10+. Estimated based on survey results for urban population.
ARG	IT.NET.USER.ZS	YR2012	Source: ITU estimate.
ARG	IT.NET.USER.ZS	YR2013	Source: ITU estimate.
ARG	IT.NET.USER.ZS	YR2014	Source: ITU estimate.
ARG	IT.NET.USER.ZS	YR2015	Source: Instituto Nacional de Estadistica y Censos.
ARG	IT.NET.USER.ZS	YR2016	Source: NATIONAL INSTITUTE OF STATISTICS AND CENSUS (INDEC)
ARG	IT.NET.USER.ZS	YR2017	Source: NATIONAL INSTITUTE OF STATISTICS AND CENSUS (INDEC)
ARG	SE.ADT.LITR.ZS	YR2006	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2009	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2010	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2011	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2012	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2013	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2014	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2015	UIS Estimation
ARG	SE.ADT.LITR.ZS	YR2016	UIS Estimation
ARG	SE.ENR.PRSC.FM.ZS	YR1980	UIS Estimation
ARG	SE.ENR.PRSC.FM.ZS	YR1982	UIS Estimation
ARG	SH.DYN.MORT	YR1969	Uncertainty bound is 68.5 - 76.8
ARG	SH.DYN.MORT	YR1970	Uncertainty bound is 68.9 - 74.7
ARG	SH.DYN.MORT	YR1971	Uncertainty bound is 68 - 73.7
ARG	SH.DYN.MORT	YR1972	Uncertainty bound is 66.4 - 73
ARG	SH.DYN.MORT	YR1973	Uncertainty bound is 64.7 - 71.7
ARG	SH.DYN.MORT	YR1974	Uncertainty bound is 62.8 - 69.5
ARG	SH.DYN.MORT	YR1975	Uncertainty bound is 60.5 - 66.1
ARG	SH.DYN.MORT	YR1976	Uncertainty bound is 57.7 - 62
ARG	SH.DYN.MORT	YR1977	Uncertainty bound is 54.1 - 57.3
ARG	SH.DYN.MORT	YR1978	Uncertainty bound is 50 - 52.3
ARG	SH.DYN.MORT	YR1979	Uncertainty bound is 45.8 - 47.7
ARG	SH.DYN.MORT	YR1980	Uncertainty bound is 42 - 43.7
ARG	SH.DYN.MORT	YR1981	Uncertainty bound is 38.7 - 40.2
ARG	SH.DYN.MORT	YR1982	Uncertainty bound is 35.8 - 37.2
ARG	SH.DYN.MORT	YR1983	Uncertainty bound is 33.4 - 34.7
ARG	SH.DYN.MORT	YR1984	Uncertainty bound is 31.5 - 32.7
ARG	SH.DYN.MORT	YR1985	Uncertainty bound is 30 - 31.2
ARG	SH.DYN.MORT	YR1986	Uncertainty bound is 29.1 - 30.2
ARG	SH.DYN.MORT	YR1987	Uncertainty bound is 28.5 - 29.7
ARG	SH.DYN.MORT	YR1988	Uncertainty bound is 28.4 - 29.5
ARG	SH.DYN.MORT	YR1989	Uncertainty bound is 28.3 - 29.4
ARG	SH.DYN.MORT	YR1990	Uncertainty bound is 28.1 - 29.2
ARG	SH.DYN.MORT	YR1991	Uncertainty bound is 27.6 - 28.6
ARG	SH.DYN.MORT	YR1992	Uncertainty bound is 26.8 - 27.9
ARG	SH.DYN.MORT	YR1993	Uncertainty bound is 25.8 - 26.8
ARG	SH.DYN.MORT	YR1994	Uncertainty bound is 24.7 - 25.7
ARG	SH.DYN.MORT	YR1995	Uncertainty bound is 23.7 - 24.6
ARG	SH.DYN.MORT	YR1996	Uncertainty bound is 22.6 - 23.5
ARG	SH.DYN.MORT	YR1997	Uncertainty bound is 21.7 - 22.5
ARG	SH.DYN.MORT	YR1998	Uncertainty bound is 20.8 - 21.6
ARG	SH.DYN.MORT	YR1999	Uncertainty bound is 20 - 20.7
ARG	SH.DYN.MORT	YR2000	Uncertainty bound is 19.3 - 20
ARG	SH.DYN.MORT	YR2001	Uncertainty bound is 18.6 - 19.3
ARG	SH.DYN.MORT	YR2002	Uncertainty bound is 18 - 18.7
ARG	SH.DYN.MORT	YR2003	Uncertainty bound is 17.3 - 18
ARG	SH.DYN.MORT	YR2004	Uncertainty bound is 16.7 - 17.4
ARG	SH.DYN.MORT	YR2005	Uncertainty bound is 16.2 - 16.9
ARG	SH.DYN.MORT	YR2006	Uncertainty bound is 15.8 - 16.4
ARG	SH.DYN.MORT	YR2007	Uncertainty bound is 15.4 - 16
ARG	SH.DYN.MORT	YR2008	Uncertainty bound is 15 - 15.6
ARG	SH.DYN.MORT	YR2009	Uncertainty bound is 14.6 - 15.2
ARG	SH.DYN.MORT	YR2010	Uncertainty bound is 14.2 - 14.7
ARG	SH.DYN.MORT	YR2011	Uncertainty bound is 13.6 - 14.1
ARG	SH.DYN.MORT	YR2012	Uncertainty bound is 13 - 13.5
ARG	SH.DYN.MORT	YR2013	Uncertainty bound is 12.4 - 12.9
ARG	SH.DYN.MORT	YR2014	Uncertainty bound is 11.8 - 12.3
ARG	SH.DYN.MORT	YR2015	Uncertainty bound is 11.2 - 11.7
ARG	SH.DYN.MORT	YR2016	Uncertainty bound is 10.7 - 11.1
ARG	SH.DYN.MORT	YR2017	Uncertainty bound is 10.1 - 10.6
ARG	SH.DYN.MORT	YR2018	Uncertainty bound is 9.5 - 10.1
ARG	SH.DYN.MORT	YR2019	Uncertainty bound is 8.8 - 9.8
ARG	SH.STA.OWAD.ZS	YR1975	Plausible bound is 33.3 - 46.8
ARG	SH.STA.OWAD.ZS	YR1976	Plausible bound is 34 - 47.1
ARG	SH.STA.OWAD.ZS	YR1977	Plausible bound is 34.9 - 47.4
ARG	SH.STA.OWAD.ZS	YR1978	Plausible bound is 35.6 - 47.7
ARG	SH.STA.OWAD.ZS	YR1979	Plausible bound is 36.4 - 48
ARG	SH.STA.OWAD.ZS	YR1980	Plausible bound is 37.3 - 48.4
ARG	SH.STA.OWAD.ZS	YR1981	Plausible bound is 38 - 48.8
ARG	SH.STA.OWAD.ZS	YR1982	Plausible bound is 38.7 - 49.2
ARG	SH.STA.OWAD.ZS	YR1983	Plausible bound is 39.5 - 49.6
ARG	SH.STA.OWAD.ZS	YR1984	Plausible bound is 40.3 - 50.1
ARG	SH.STA.OWAD.ZS	YR1985	Plausible bound is 41 - 50.6
ARG	SH.STA.OWAD.ZS	YR1986	Plausible bound is 41.7 - 51.1
ARG	SH.STA.OWAD.ZS	YR1987	Plausible bound is 42.4 - 51.6
ARG	SH.STA.OWAD.ZS	YR1988	Plausible bound is 43.1 - 52.2
ARG	SH.STA.OWAD.ZS	YR1989	Plausible bound is 43.8 - 52.6
ARG	SH.STA.OWAD.ZS	YR1990	Plausible bound is 44.5 - 53.2
ARG	SH.STA.OWAD.ZS	YR1991	Plausible bound is 45.1 - 53.7
ARG	SH.STA.OWAD.ZS	YR1992	Plausible bound is 45.8 - 54.2
ARG	SH.STA.OWAD.ZS	YR1993	Plausible bound is 46.4 - 54.7
ARG	SH.STA.OWAD.ZS	YR1994	Plausible bound is 47.1 - 55.2
ARG	SH.STA.OWAD.ZS	YR1995	Plausible bound is 47.7 - 55.7
ARG	SH.STA.OWAD.ZS	YR1996	Plausible bound is 48.3 - 56.2
ARG	SH.STA.OWAD.ZS	YR1997	Plausible bound is 48.9 - 56.7
ARG	SH.STA.OWAD.ZS	YR1998	Plausible bound is 49.5 - 57.2
ARG	SH.STA.OWAD.ZS	YR1999	Plausible bound is 50.2 - 57.7
ARG	SH.STA.OWAD.ZS	YR2000	Plausible bound is 50.7 - 58.1
ARG	SH.STA.OWAD.ZS	YR2001	Plausible bound is 51.3 - 58.6
ARG	SH.STA.OWAD.ZS	YR2002	Plausible bound is 51.9 - 59.1
ARG	SH.STA.OWAD.ZS	YR2003	Plausible bound is 52.5 - 59.6
ARG	SH.STA.OWAD.ZS	YR2004	Plausible bound is 53 - 60.1
ARG	SH.STA.OWAD.ZS	YR2005	Plausible bound is 53.6 - 60.6
ARG	SH.STA.OWAD.ZS	YR2006	Plausible bound is 54.1 - 61.1
ARG	SH.STA.OWAD.ZS	YR2007	Plausible bound is 54.6 - 61.6
ARG	SH.STA.OWAD.ZS	YR2008	Plausible bound is 55.1 - 62.2
ARG	SH.STA.OWAD.ZS	YR2009	Plausible bound is 55.6 - 62.7
ARG	SH.STA.OWAD.ZS	YR2010	Plausible bound is 56.1 - 63.3
ARG	SH.STA.OWAD.ZS	YR2011	Plausible bound is 56.5 - 63.9
ARG	SH.STA.OWAD.ZS	YR2012	Plausible bound is 56.9 - 64.6
ARG	SH.STA.OWAD.ZS	YR2013	Plausible bound is 57.2 - 65.2
ARG	SH.STA.OWAD.ZS	YR2014	Plausible bound is 57.5 - 65.9
ARG	SH.STA.OWAD.ZS	YR2015	Plausible bound is 57.8 - 66.6
ARG	SH.STA.OWAD.ZS	YR2016	Plausible bound is 58.1 - 67.2
ARG	SI.DST.FRST.20	YR1980	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1986	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1987	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1991	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1992	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1993	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1994	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1995	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1996	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1997	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1998	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR1999	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2000	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2001	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2002	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2003	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2004	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2005	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2006	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2007	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2008	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2009	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2010	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2011	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2012	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2013	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2014	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2016	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2017	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.DST.FRST.20	YR2018	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1980	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1986	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1987	Based on data from EPH. Estimated from grouped income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1991	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1992	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1993	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1994	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1995	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1996	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1997	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1998	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR1999	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2000	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2001	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2002	Based on data from EPH. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2003	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2004	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2005	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2006	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2007	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2008	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2009	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2010	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2011	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2012	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2013	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2014	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2016	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2017	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.GINI	YR2018	Based on data from EPHC-S2. Estimated from unit-record income data. Urban only. PPP is regression based.
ARG	SI.POV.NAHC	YR2016	Source: Instituto Nacional de Estadisticas y Censos (INDEC), EPH-C, Second semester. Poverty line is absolute. World Bank estimate.
ARG	SI.POV.NAHC	YR2017	Source: Instituto Nacional de Estadisticas y Censos (INDEC), EPH-C, Second semester. Poverty line is absolute. World Bank estimate.
ARG	SI.POV.NAHC	YR2018	Source: Instituto Nacional de Estadisticas y Censos (INDEC), EPH-C, Second semester. Poverty line is absolute. World Bank estimate.
ARG	SI.POV.NAHC	YR2019	Source: Instituto Nacional de Estadisticas y Censos (INDEC), EPH-C, Second semester. Poverty line is absolute. World Bank estimate.
ARG	SI.SPR.PCAP.ZG	YR2017	Estimated from individual income data. Growth rates are for the period of 2012-2017.
ARG	SI.SPR.PCAP.ZG	YR2018	Estimated from income data. Growth rates are for the period of 2013-2018.
ARG	SL.TLF.0714.ZS	YR2004    	Encuesta sobre Actividades de Niños, Niñas y Adolescentes
ARG	SL.TLF.0714.ZS	YR2012	Módulo de Actividades de Niños, Niñas y Adolescentes (MANNyA)
ARG	SN.ITK.DEFC.ZS	YR2014	Proportion of undernourished less than 2.5 percent.

"""

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.create_documents([doc_txt])

In [23]:
output = chain({"input_documents": docs, 
                "country_name": "Argentina"
               })

ValueError: OpenAIChat currently only supports single prompt, got ["\nYou are tasked to extract ESG (Environmental, Social, and Governance) topics for each country records. You will be provided with country name Argentina  and text ARG\tIT.NET.USER.ZS\tYR2000\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2001\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2002\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2003\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2004\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2005\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2006\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2007\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2008\tSource: SECOM.\nARG\tIT.NET.USER.ZS\tYR2009\tSource: Comisión Nacional de Comunicaciones\nARG\tIT.NET.USER.ZS\tYR2010\tSource: ITU estimate.\nARG\tIT.NET.USER.ZS\tYR2011\tSource: ITU estimate based on INDEC's survey.;Note: Population aged 10+. Estimated based on survey results for urban population.\nARG\tIT.NET.USER.ZS\tYR2012\tSource: ITU estimate.\nARG\tIT.NET.USER.ZS\tYR201